In [3]:
from transformers import VideoMAEImageProcessor, VideoMAEForPreTraining
import numpy as np
import torch

# Number of frames in the video
num_frames = 16

# Generate a random video and normalize the values to [0, 1]
video = np.random.randn(num_frames, 3, 224, 224)
video = (video - video.min()) / (video.max() - video.min())
video = list(video)

# Initialize the processor and the model
processor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEForPreTraining.from_pretrained("MCG-NJU/videomae-base")

# Process the video frames to get pixel values
pixel_values = processor(video, return_tensors="pt").pixel_values

# Calculate the number of patches per frame and the sequence length
num_patches_per_frame = (model.config.image_size // model.config.patch_size) ** 2
seq_length = (num_frames // model.config.tubelet_size) * num_patches_per_frame

# Generate random boolean masked positions
bool_masked_pos = torch.randint(0, 2, (1, seq_length)).bool()

# Pass the pixel values and masked positions to the model
outputs = model(pixel_values, bool_masked_pos=bool_masked_pos)

# Extract the loss from the outputs
loss = outputs.loss

# Print the loss
print("Loss:", loss.item())


It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
C:\Users\kitmi\anaconda3\Lib\site-packages\transformers\feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  return torch.tensor(value)


Loss: 0.993100106716156


In [4]:
# Evaluation code
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in dataloader:
        videos, labels = batch
        pixel_values = processor(videos, return_tensors="pt").pixel_values

        outputs = model(pixel_values)
        _, predicted = torch.max(outputs.logits, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

NameError: name 'dataloader' is not defined